## train_test_split,KFold,StratifiedKFold,cross_validation.StratifiedShuffleSplit

## cross_val_score(model,x,y,cv)

##  grid_search.RandomizedSearchCV(model, parameters_grid, scoring, cv = cv),GridSearchCV(model,param_grid,cv)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, auc, roc_curve,accuracy_score,confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
x1,x2,y1,y2=train_test_split(x,y,random_state=0,train_size=0.5)
model.fit(x,y)
y2_model=model.predict(x2)
accuracy_score(y2,y2_model)
mat=confusion_matrix(y2,y2_model)

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(model,x,y,cv=5)

In [ ]:
from sklearn.model_selection import LeaveOneOut
scores=cross_val_score(model,x,y,cv=LeaveOneOut(len(x)))
scores
scores.mean()

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=5)
i = 0
for train_index, test_index in kf.split(X):
    i +=1 
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("FOLD {}: ".format(i))
    print("X_test: ", X_test)
    print("Y_test: ", y_test)
    print("-"*30)

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
kfold=StratifiedKFold(y=y_train,n_folds=10,random_state=1)
scores=[]
for k,(train,test) in enumerate(kfold):
    pipe_lr.fit(x_train[train],y_train[train])
    score=pipe_lr.score(x_train[test],y_train[test])
    scores.append(score)
    print('Fold:%s, Class dist.:%s,Acc:%.3f' % (k+1,np.bincount(y_train[train]),score))

## RandomizedSearchCV+StratifiedShuffleSplit

In [3]:
from sklearn.model_selection import StratifiedShuffleSplit

In [5]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
model = xgb.XGBClassifier()

In [ ]:
parameters_grid = { 'learning_rate' : [0.1, 0.2, 0.5], 'max_depth' : [5, 10, 15], 'n_estimators' : [150, 250, 300], 'min_child_weight' : [3, 5, 10] }

In [6]:
cv = StratifiedShuffleSplit(train_labels, n_iter = 3, test_size = 0.3)

NameError: name 'train_labels' is not defined

In [4]:
grid_cv = RandomizedSearchCV(model, parameters_grid, scoring = 'accuracy', cv = cv)

NameError: name 'grid_search' is not defined

In [ ]:
grid_cv.fit(train_data, train_labels)

In [ ]:
grid_cv.bestestimator

In [ ]:
model = xgb.XGBClassifier(learning_rate=0.1, max_depth=15, min_child_weight=5, n_estimators=250)
model.fit(train_data, train_labels)

## 驗證曲線，學習曲線

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
def PolynomialRegression(degree=2,**kwargs):
    return make_pipeline(PolynomialFeatures(degree),LinearRegression(**kwargs))

In [ ]:
from sklearn.learning_curve import validation_curve
degree=np.arange(0,21)
train_score,val_score=validation_curve(PolynomialRegression(),x,y,'polynomialfeatures__degree',degree,cv=7)
plt.plot(degree,np.median(train_score,1),color='blue',label='training score')
plt.plot(degree,np.median(val_score,1),color='red',label='validation score')
plt.legend(loc='best')
plt.ylim(0,1)
plt.xlabel('degree')
plt.ylabel('score')


In [ ]:
from sklearn.learning_curve import validation_curve
param_range=[0.001,0.01,0.1,1.0,10.0,100.0]
train_scores,test_scores=validation_curve(estimator=pipe_lr,x=x_train,y=y_train,param_name='clf_C',param_range=param_range,cv=10)
train_mean=np.mean(train_scores,axis=1)
train_std=np.std(train_scores,axis=1)
test_mean=np.mean(test_scores,axis=1)
test_std=np.std(test_scores,axis=1)
plt.plot(params_range,train_mean,color='blue',marker='o',markersize=5,label='training accuracy')
plt.fill_between(param_range,train_mean+train_std,train_mean-train_std,alpha=0.15,color='blue')
plt.plot(param_range,test_mean,color='green',linestyle='--',marker='s',markersize=5,label='validation accuracy')
plt.fill_between(param_range,test_mean+test_std,test_mean-test_std,alpha=0.15,color='green')
plt.grid()
plt.xscale('log')
plt.legend(loc='lower right')
plt.xlabel('Parameter C')
plt.ylabel('Accuracy')
plt.ylim([0.8,1.0])
plt.show()

In [ ]:
from sklearn.learning_curve import learning_curve
fig,ax=plt.subplots(1,2,figsize=(16,6))
fig.subplots_adjust(left=0.0625,right=0.95,wspace=0.1)

for i,degree in enumerate([2,9]):
    N,train_lc,val_lc=learning_curve(PolynomialRegression(degree),x,y,cv=7,train_sizes=np.linspace(0.3,1,25))
    ax[i].plot(N,np.mean(train_lc,1),color='blue',label='training score')
    ax[i].plot(N,np.mean(val_lc,1),color='red',label='validation score')
    ax[i].hlines(np.mean([train_lc[-1],val_lc[-1]]),N[0],N[-1],color='gray',linestyle='dashed')
    ax[i].set_ylim(0,1)
    ax[i].set_xlim(N[0],N[-1])
    ax[i].set_xlabel('training size')
    ax[i].set_ylabel('score')
    ax[i].set_title('degree={0}'.format(degree),size=14)
    ax[i].legend(loc='best')

## Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV 
param_grid={'polynomialfeatures__degree':np.arange(21),'linearregression__fit__intercept':[True,False],'linearregression__normalize':[True,False]}
grid=GridSearchCV(PolynomialRegression(),param_grid,cv=7)
grid.fit(x,y)
grid.best_params_
model=grid.best_estimator_
plt.scatter(x.ravel(),y)
lim=plt.axis()
y_test=model.fit(x,y).predict(x_test)
plt.plot(x_test.ravel(),y_test,hold=True)
plt.axis(lim)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
pipe_svc=Pipeline([('scl',StandardScaler()),('clf',SVC(random_state=1))])
param_range=[0.0001,0.001,0.01,0.1,1.0,10.0,100.0,1000.0]
param_grid=[{'clf_C':param_range,'clf_kernel':['linear']},{'clf_C':param_range,'clf_gamma':param_range,'clf_kernel':['rbf']}]
gs=GridSearchCV(estimator=pipe_svc,param_grid=param_grid,scoring='accuracy',cv=10,n_jobs=-1)
gs=gs.fit(x_train,y_train)
print(gs.best_score_)
print(gs.best_params_)
clf=gs.best_estimator_
clf.fit(x_train,y_train)
print('Test Accuracy: %.3f' % clf.score(x_test,y_test))

In [ ]:

# 設定要訓練的超參數組合
n_estimators = [50, 100, 150]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(reg, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

In [ ]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
scores=[val.mean_validation_score for val in grid.grid_scores_]


In [ ]:
plt.semilogx(bandwidths,scores)
plt.xlabel('bandwidth')
plt.ylabel('accuracy')
plt.title('KDE Model Performance')
print(grid.best_params_)
print('accuracy=',grid.best_score_)

In [ ]:
# 定義 PCA 與隨後的邏輯斯迴歸函數
logistic = SGDClassifier(loss='log', penalty='l2', max_iter=10000, tol=1e-5, random_state=0)
pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

# 載入手寫數字辨識集
digits = datasets.load_digits()
X_digits = digits.data
y_digits = digits.target

In [ ]:
# 先執行 GridSearchCV 跑出最佳參數
param_grid = {
    'pca__n_components': [4, 10, 20, 30, 40, 50, 64],
    'logistic__alpha': np.logspace(-4, 4, 5),
}
search = GridSearchCV(pipe, param_grid, iid=False, cv=5)
search.fit(X_digits, y_digits)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

## 評估分類

In [ ]:
from sklearn.metrics import confusion_matrix
pipe_svc.fit(x_train,y_train)
y_pred=pipe_svc.predict(x_test)
confmat=confusion_matrix(y_true=y_test,y_pred=y_pred)
print(confmat)

In [ ]:
fig,ax=plt.subplots(figsize=(2.5,2.5))
ax.matshow(confmat,cmap=plt.cm.Blues,alpha=0.3)
for i in range(confmat[0]):
    for j in range(confmat[1]):
        ax.text(x=j,y=i,s=confmat[i,j],va='center',ha='center')
plt.xlabel('predicted label')
plt.ylabel('true label')
plt.show()

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score,f1_score
print('Precision:%.3f' % precision_score(y_true=y_test,y_pred=y_pred))
print('Recall:%.3f' % recall_score(y_true=y_test,y_pred=y_pred))
print('F1:%.3f' % f1_score(y_true=y_test,y_pred=y_pred))

In [ ]:
from sklearn.metrics import make_scorer,f1_score
scorer=make_scorer(f1_score,pos_label=0)
gs=GridSearchCV(estimator=pipe_svc,param_grid=param_grid,scoring=scorer,cv=10)

In [ ]:
from sklearn.metrics import roc_curve,auc
from scipy import interp
x_train2=x_train[:,[4,14]]
cv=StratifiedKFold(y_train,n_folds=3,random_state=1)
fig=plt.figure(figsize=(7,5))
mean_tpr=0.0
mean_fpr=np.linspace(0,1,100)
all_tpr=[]
for i,(train,test) in enumerate(cv):
    probas=pipe_lr.fit(x_train2[train])
    y_train[train].predict_proba(x_train2[test])
    fpr,tpr,threshold=roc_curve(y_train[test],probas[:,1],pos_label=1)
    mean_tpr+=interp(mean_fpr,fpr,tpr)
    mean_tpr[0]=0.0
    roc_auc=auc(fpr,tpr)
    plt.plot(fpr,tpr,lw=1,label='ROC fold %d (area=%0.2f)' % (i+1,roc_auc))
plt.plot([0,1],[0,1],linestyle='--',color=(0.6,0.6,0.6),label='random guessing')
mean_tpr/=len(cv)
mean_tpr[-1]=1.0
mean_auc=auc(mean_fpr,mean_tpr)
plt.plot(mean_fpr,mean_tpr,'k--',label='mean ROC (area=%0.2f)' % mean_auc,lw=2)
plt.plot([0,0,1],[0,1,1],lw=2,linestyle=':',color='black',label='perfect performance')
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.title('Receiver Operator Characteristic')
plt.legend(loc='lower right')
plt.show()


In [ ]:
pipe_svc=pipe_svc.fit(x_train2,y_train)
y_pred2=pipe_svc.predict(x_test[:,[4,14]])
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
print('ROC AUC:%.3f' % roc_auc_score(y_true=y_test,y_score=y_pred2))
print('Accuracy:%.3f' % accuracy_score(y_true=y_test,y_pred=y_pred2))

## 多分類評分

In [ ]:
pre_scorer=make_scorer(score_func=precision_score,pos_label=1,greater_is_better=True,average='micro')